In [3]:
# Importing libraries

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import pickle
import os
import re
import joblib

In [4]:
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)
pd.set_option("display.float_format", '{:.2f}'.format)

##### Utils Functions

In [5]:
## Imputation
# Fill missing values with Unknown
def FillWithUnknown(data, cols):
    if isinstance(cols, list):
        for col in cols:
            data[col] = data[col].fillna("Unknown")
        
    elif isinstance(cols, str):
        data[cols] = data[cols].fillna("Unknown")

# Sort and Ffill
def FillAfterSorting(data, cols, sorting_col=""):
    if sorting_col:
        data.sort_values(by=sorting_col, inplace=True)
        if isinstance(cols, list):
            for col in cols:
                data[col] = data[col].ffill().bfill()
        
        elif isinstance(cols, str):
            data[cols] = data[cols].ffill().bfill()


In [6]:
# Remove outliers using IQR
def RemoveOutliers(data, cols):
    for col in cols:
        q1, q2, q3 = data.loc[:, col].quantile([0.25, 0.5, 0.75])
        iqr = q3 - q1
        lower_threshold = round(q1 - (1.5 * iqr), 2)
        upper_threshold = round(q3 + (1.5 * iqr), 2)
        median = data.loc[:, col].median()
        data.loc[ (data[col] < lower_threshold) | (data[col] > upper_threshold), col] = median

In [7]:
## Feature Engineering 
def ConvertRoomsToNumber(value):
    
    if 'B/R' in value:
        return int(re.findall('\d+', value)[0])  # Extract the number before "B/R"
    elif 'STUDIO' in value or 'UNKNOWN' in value:
        return 0  # Represent Studio as 0 bedrooms
    elif value in ['OFFICE', 'SHOP', 'PENTHOUSE', 'SINGLE ROOM', 'HOTEL']:
        return -1  # Assign -1 or another placeholder for non-bedroom categories
    elif 'SINGLE ROOM' in value:
        return 1
    else:
        return np.nan  # Handle unexpected values

def TransactionAgeInDays(data, col):
    current_time = pd.Timestamp.now()
    data['transaction_age_in_days'] = round((current_time - pd.to_datetime(data[col])).dt.total_seconds() / (60 * 60 * 24))

def RoomsCleaner(df, col):
    df[col] = df[col].str.upper()
    df[col] = df[col].apply(ConvertRoomsToNumber)

In [8]:
## One Hot Encoding
def ApplyOneHotEncoder(df, cols, save_path="encoders"):
    ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    encoded_data = ohe.fit_transform(df[cols])
    encoded_df = pd.DataFrame(encoded_data, columns=ohe.get_feature_names_out(df[cols].columns))
    df.drop(cols, axis=1, inplace=True)
    df = pd.concat([df, encoded_df], axis=1)
    df.reset_index(drop=True, inplace=True)
    os.makedirs(save_path, exist_ok=True)
    joblib.dump(ohe, f"{save_path}/OneHotEncoder.pkl")
    return df

## Encoding Features
def ApplyLabelEncoder(df, cols, save_path="encoders"):
    os.makedirs(save_path, exist_ok=True)

    label_encoders = {}
    for col in cols:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])
        label_encoders[col] = le

    # Save the label encoder
    for col, le in label_encoders.items():
        joblib.dump(le, f"{save_path}/LabelEncoder_{col}.pkl")


In [9]:
def ScaleData(df, columns_to_scale, save_path="scaler.pkl"):
    scaler = StandardScaler()
    df[columns_to_scale] = scaler.fit_transform(df[columns_to_scale])
    with open(save_path, "wb") as f:
        pickle.dump(scaler, f)

#### Exploratory Data Analysis

In [10]:
# Explore Datasets
rents_df = pd.read_csv("../data/snp_dld_2024_rents.csv", low_memory=False)
transactions_df = pd.read_csv("../data/snp_dld_2024_transactions.csv",)

##### EDA snp_dld_2024_transactions.csv

In [11]:
# transactions_df.info(show_counts=True)
# transactions_df.filter(regex="^(?!.*_ar$).*$").head(5)
# transactions_df.head(5)

In [12]:
# Filter columns after manual analysis
OHE_cols = ["transaction_type_en", "registration_type_en", "is_freehold_text", "property_usage_en", "is_offplan",
             "is_freehold", "nearest_landmark_en", "property_type_en", "rooms_en", "nearest_mall_en"]

LE_cols =   ["transaction_subtype_en", "property_subtype_en", "area_en", "nearest_metro_en"]

target_col = ['amount']

df = transactions_df.loc[:, OHE_cols + LE_cols + ["transaction_datetime"] + target_col]

In [13]:
# Imputation
FillAfterSorting(df, ["is_freehold", "nearest_landmark_en", "nearest_mall_en", "nearest_metro_en"], 'area_en')
FillWithUnknown(df, ['property_subtype_en', 'rooms_en'])

In [14]:
# Feature Engineering
RoomsCleaner(df, 'rooms_en')
TransactionAgeInDays(df, 'transaction_datetime')
df.drop(['transaction_datetime'], axis=1, inplace=True)


In [15]:
# Encoding Categorical Columns
ONE_HOT_COLUMNS = []
LE_COLUMNS = []
for col in df.select_dtypes('object').columns:
    if df[col].nunique() <= 10:
        ONE_HOT_COLUMNS.append(col)
    else:
        LE_COLUMNS.append(col)

ApplyLabelEncoder(df, LE_COLUMNS)
final_df = ApplyOneHotEncoder(df, ONE_HOT_COLUMNS)


In [16]:
## Remove Outliers
RemoveOutliers(final_df, ["amount", "transaction_age_in_days"])

## Scale Dataset
ScaleData(final_df, df.drop('amount', axis=1).columns.tolist())


In [ ]:
## Correlation Analysis
def FeatureSelectionCorr(final_df):
    corr_df = final_df.corr(method='pearson')
    COLUMNS = corr_df[(corr_df['amount']>0.01).round(2) | (corr_df['amount'].round(2)<-0.01)]['amount'].index.tolist()
    COLUMNS.remove('amount')
    return

COLUMNS = FeatureSelectionCorr(final_df)
TARGET = "amount"


In [18]:
X_train, X_test, y_train, y_test = train_test_split(final_df[COLUMNS], final_df[TARGET], random_state=104, test_size=0.1, shuffle=True)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=104, test_size=0.1, shuffle=True)


In [19]:
def evaluate_model(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    return {"RMSE" : round(rmse, 2), "R2 Score": round(r2, 2), "MAE": round(mae, 2)}

In [18]:
from xgboost import XGBRegressor

def TrainXGBRegressor(model_parameters):
    xgb = XGBRegressor(**model_parameters)
    xgb.fit(X_train, y_train)
    xgb_preds = xgb.predict(X_val)
    xgb_metrics = evaluate_model(y_val, xgb_preds)
    return xgb_metrics

{'RMSE': np.float64(590410.62), 'R2 Score': 0.72, 'MAE': np.float64(356801.66)}

In [19]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state=42)
rf.fit(X_train, y_train)
rf_preds = rf.predict(X_val)
rf_metrics = evaluate_model(y_val, rf_preds)
rf_metrics

{'RMSE': np.float64(596917.61), 'R2 Score': 0.71, 'MAE': np.float64(321926.51)}

In [ ]:
# NOTE: It is taking too much time to train
# from sklearn.svm import SVR
# from sklearn.pipeline import make_pipeline

# svr = make_pipeline(StandardScaler(), SVR())
# svr.fit(X_train, y_train)
# svr_preds = svr.predict(X_val)
# svr_metrics = evaluate_model(y_val, svr_preds)
# svr_metrics

In [20]:
# !pip install optuna

In [ ]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

def find_best_params(X_train, y_train, param_grid, cv=3, scoring='neg_mean_squared_error', verbose=2):

    xgb = XGBRegressor(random_state=42)
    grid_search = GridSearchCV(
        estimator=xgb,
        param_grid=param_grid,
        cv=cv,
        scoring=scoring,
        verbose=verbose
    )
    
    grid_search.fit(X_train, y_train)
    return grid_search.best_params_

# Example usage:
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 10],
    'learning_rate': [0.01, 0.1, 0.2]
}

best_params = find_best_params(X_train, y_train, param_grid)
print("Best parameters:", best_params)


Fitting 3 folds for each of 27 candidates, totalling 81 fits
[CV] END ...learning_rate=0.01, max_depth=3, n_estimators=50; total time=   0.4s
[CV] END ...learning_rate=0.01, max_depth=3, n_estimators=50; total time=   0.2s
[CV] END ...learning_rate=0.01, max_depth=3, n_estimators=50; total time=   0.2s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=100; total time=   0.4s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=100; total time=   0.4s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=100; total time=   0.4s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=200; total time=   0.8s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=200; total time=   0.8s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=200; total time=   0.9s
[CV] END ...learning_rate=0.01, max_depth=5, n_estimators=50; total time=   0.3s
[CV] END ...learning_rate=0.01, max_depth=5, n_estimators=50; total time=   0.3s
[CV] END ...learning_rate=0.01, max_depth=5, n_e

In [ ]:
def TrainXGBRegressor(model_parameters):
    xgb = XGBRegressor(**model_parameters)
    xgb.fit(X_train, y_train)
    xgb_preds = xgb.predict(X_val)
    xgb_metrics = evaluate_model(y_val, xgb_preds)
    return xgb_metrics

params = {'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 100, "random_state": 42}
xgb_metrics = TrainXGBRegressor(params)
print(xgb_metrics)

In [27]:
### Drop
    # [transaction_number, transaction_datetime, transaction_type_id, transaction_size_sqm, transaction_subtype_id, property_usage_id,
    #  property_id, property_type_ar, property_type_id, building_age, rooms_ar, project_name_en, property_subtype_ar,
    #  property_subtype_id, area_ar, area_id, nearest_landmark_ar, nearest_metro_ar, nearest_mall_ar, master_project_ar, req_from,
    #  req_to, entry_id, meta_ts, master_project_en, project_name_en]


### Encoding
    ## Label Encoding
    # ["transaction_subtype_en", "property_subtype_en", "area_en", "project_name_en", "nearest_metro_en"]

    ## One Hot Encoding
    ## ["transaction_type_en", "registration_type_en", "is_freehold_text", "property_usage_en", "is_offplan", "is_freehold",
    #  "nearest_landmark_en", "property_type_en", "rooms_en", "nearest_mall_en"]

### Engineered Features
    # [transaction_age_in_days] -> transactions_df.loc[:, 'transaction_age_in_days'] = round((pd.Timestamp.now() - pd.to_datetime(transactions_df['transaction_datetime'])).dt.total_seconds() / (60 * 60 * 24))

# transactions_df['property_size_sqm'].plot(kind='box', title='Property Size SQM')

157600

##### EDA snp_dld_2024_rents.csv

In [23]:
rents_df.head(10)

,ejari_contract_number,registration_date,contract_start_date,contract_end_date,version_number,version_text,contract_amount,annual_amount,is_freehold,is_freehold_text,property_size_sqm,parcel_id,property_id,land_property_id,property_type_en,property_type_ar,property_subtype_en,property_subtype_ar,property_usage_en,property_usage_ar,property_usage_id,total_properties,rooms,parking,project_name_en,project_name_ar,area_en,area_ar,area_id,nearest_landmark_en,nearest_landmark_ar,nearest_metro_en,nearest_metro_ar,nearest_mall_en,nearest_mall_ar,master_project_en,master_project_ar,ejari_property_type_id,ejari_property_sub_type_id,req_from,req_to,entry_id,meta_ts
0,120130625001365,2024-01-24 11:14:33,2024-01-01,2024-12-31,12,Renewed,58000.00,58000.0,f,Non Free Hold,120.76,2410596.0,0,0,Unit,وحدة,Flat,شقه,Residential,سكني,0,1,NaN,NaN,NaN,NaN,Al Nahda Second,النهده الثانيه,0,Dubai International Airport,مطار دبي الدولي,Al Nahda Metro Station,محطة مترو النهضة,City Centre Mirdif,سيتي سنتر مردف,NaN,NaN,0,0,2024-01-01,2024-01-31,94368,2024-09-26 11:18:46.859
1,120130626005726,2024-01-09 16:48:47,2024-03-18,2025-03-17,12,Renewed,88000.00,88000.0,t,Free Hold,159.03,2514114.0,0,0,Unit,وحدة,Flat,شقه,Residential,سكني,0,1,NaN,NaN,NaN,NaN,Mirdif,مردف,0,Dubai International Airport,مطار دبي الدولي,Rashidiya Metro Station,محطة مترو الراشدية,City Centre Mirdif,سيتي سنتر مردف,NaN,NaN,0,0,2024-01-01,2024-01-31,94368,2024-09-26 11:18:46.859
2,120130626007829,2024-01-16 11:39:26,2024-01-01,2024-12-31,12,Renewed,94000.00,94000.0,f,Non Free Hold,60.94,1230681.0,0,0,Unit,وحدة,Shop,محل,Commercial,تجاري,0,1,NaN,NaN,NaN,NaN,Al Muteena,المطينه,0,Dubai International Airport,مطار دبي الدولي,Salah Al Din Metro Station,محطة مترو صلاح الدين,Dubai Mall,مول دبي,NaN,NaN,0,0,2024-01-01,2024-01-31,94368,2024-09-26 11:18:46.859
3,120130627008661,2024-01-15 18:28:05,2024-01-01,2024-12-31,12,Renewed,66000.00,66000.0,f,Non Free Hold,108.88,3730405.0,0,0,Unit,وحدة,Flat,شقه,Residential,سكني,0,1,NaN,NaN,NaN,NaN,Al Barsha First,البرشاء الاولى,0,Burj Al Arab,برج العرب,Sharaf Dg Metro Station,محطة مترو شرف دي جي,Mall of the Emirates,مول الإمارات,NaN,NaN,0,0,2024-01-01,2024-01-31,94368,2024-09-26 11:18:46.859
4,120130627009439,2024-01-22 13:34:29,2024-01-01,2024-12-31,12,Renewed,31200.00,31200.0,f,Non Free Hold,57.60,2320537.0,0,0,Unit,وحدة,Flat,شقه,Residential,سكني,0,1,NaN,NaN,NaN,NaN,Al Qusais First,القصيص الاولى,0,Dubai International Airport,مطار دبي الدولي,Airport Free Zone,المنطقة الحرة بالمطار,City Centre Mirdif,سيتي سنتر مردف,NaN,NaN,0,0,2024-01-01,2024-01-31,94368,2024-09-26 11:18:46.859
5,120130630014940,2024-01-05 15:45:05,2024-01-08,2024-04-07,12,Renewed,10106.25,40425.0,f,Non Free Hold,106.74,2450405.0,0,0,Unit,وحدة,Flat,شقه,Residential,سكني,0,1,NaN,NaN,NaN,NaN,Muhaisanah Fourth,محيصنه الرابعه,0,Dubai International Airport,مطار دبي الدولي,Etisalat Metro Station,محطة مترو اتصالات,City Centre Mirdif,سيتي سنتر مردف,NaN,NaN,0,0,2024-01-01,2024-01-31,94368,2024-09-26 11:18:46.859
6,120130701019951,2024-01-09 16:49:15,2023-11-08,2024-02-07,14,Renewed,12750.00,51000.0,t,Free Hold,103.05,2514313.0,0,0,Unit,وحدة,Flat,شقه,Residential,سكني,0,1,NaN,NaN,NaN,NaN,Mirdif,مردف,0,Dubai International Airport,مطار دبي الدولي,Rashidiya Metro Station,محطة مترو الراشدية,City Centre Mirdif,سيتي سنتر مردف,NaN,NaN,0,0,2024-01-01,2024-01-31,94368,2024-09-26 11:18:46.859
7,120130701021228,2024-01-24 12:37:03,2023-12-21,2024-12-20,13,Renewed,96000.00,96000.0,f,Non Free Hold,148.00,3140136.0,0,0,Unit,وحدة,Flat,شقه,Residential,سكني,0,1,NaN,NaN,NaN,NaN,Um Hurair First,ام هرير الاولى,0,Dubai International Airport,مطار دبي الدولي,Burjuman Metro Station,محطة مترو برجمان,Dubai Mall,مول دبي,NaN,NaN,0,0,2024-01-01,2024-01-31,94368,2024-09-26 11:18:46.859
8,120130701021441,2024-01-04 12:27:24,2024-01-01,2024-12-31,12,Renewed,162350.00,162350.0,f,Non Free Hold,302.00,3640312.0,0,0,Unit,وحدة,Warehouse complex,مجمع مخازن,Industrial,صناعي,0,1,NaN,NaN,NaN,NaN,Al Goze Industrial First,القوز الصناعيه الاولى,0,Burj Al 

(747408, 43) (162806, 46)
